# End-to-end evaluation

<!-- ## Experiment Overview

| **Topic**                 | Description                                                                                                                                                                                                                                                                                                         |
| ------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 📝 **Hypothesis**         | Exploratory hypothesis: "Can introducing a new language model improve the system's performance?"                                                                                                                                                                                                                    |
| ⚖️ **Comparison**         | We will compare **GPT3-3.5** (from OpenAI) to **Mistral**(open-source)                                                                                                                                                                                                                                              |
| 🎯 **Evaluation Metrics** | We will look at human-centric metrics ([Groundedness, Relevance, Coherence, Similarity, Fluency](https://learn.microsoft.com/en-us/azure/machine-learning/prompt-flow/concept-model-monitoring-generative-ai-evaluation-metrics?view=azureml-api-2)) using another LLM as judge approach to compare the performance |
| 📊 **Evaluation Dataset** | 300 question-answer pairs generated from [code-with-engineering](../data/docs/code-with-engineering/) and [code-with-mlops](../data/docs/code-with-mlops/) sections from Solution Ops repository.                                                                                                                   | -->


## Overview

In this section we will evaluate end-to-end our RAG solution using human-centric metrics ([Groundedness, Relevance, Coherence, Similarity, Fluency](https://learn.microsoft.com/en-us/azure/machine-learning/prompt-flow/concept-model-monitoring-generative-ai-evaluation-metrics?view=azureml-api-2)) as well as [ROUGE](https://medium.com/nlplanet/two-minutes-nlp-learn-the-rouge-metric-by-examples-f179cc285499), which is a traditional evaluation metric.

We will use MLflow open-source tool, more precisely, the [MLflow LLM Evaluate](https://mlflow.org/docs/latest/llms/llm-evaluate/index.html#mlflow-llm-evaluate) API.

## Setup


In [2]:
%run -i ./pre-requisites.ipynb
%run -i ./helpers/search.ipynb

In [30]:
import os
azure_openai_key = os.environ["azure_openai_key"]
azure_aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
aoi_deployment_name = os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"]

os.environ.setdefault("OPENAI_API_KEY", azure_openai_key)
os.environ.setdefault("OPENAI_API_BASE", azure_aoai_endpoint)
os.environ.setdefault("OPENAI_API_VERSION", "2023-05-15")
os.environ.setdefault("OPENAI_API_TYPE", "azure")
os.environ.setdefault("OPENAI_DEPLOYMENT_NAME", aoi_deployment_name)

import mlflow
import openai
import os
import pandas as pd
from getpass import getpass
from azureml.core import Workspace

'chat'

In [4]:
from openai import AzureOpenAI


def call_llm(messages: list[dict]):
    client = AzureOpenAI(
        api_key=azure_openai_key,
        api_version="2023-07-01-preview",
        azure_endpoint=azure_aoai_endpoint
    )

    response = client.chat.completions.create(
        model=azure_openai_chat_deployment, messages=messages)
    return response.choices[0].message.content

In [5]:
def rag(query, search_index_name, embedding_function):
    query_embeddings = embedding_function(query)

    # 1. Search for relevant documents
    search_response = search_documents(
        query_embeddings, search_index_name, embedding_function)
    # 2. Create prompt with the query, retrieved documents
    prompt_from_chunk_context = create_prompt(query, search_response)

    # 3. Call the Azure OpenAI GPT model
    response = call_llm(prompt_from_chunk_context)

## Function to generate answers for each question in evaluation dataset


In [75]:
import json
import os
%run -i ./helpers/search.ipynb

def generate_answers_for_qa(evaluation_data_path, search_index_name, embedding_function, path_to_output):
    try:
        with open(evaluation_data_path, "r", encoding="utf-8") as file:
            evaluation_data = json.load(file)
            generated_qa = []
            for data in evaluation_data:
                question = data["user_prompt"]

                # 1. Search in the index
                search_response = search_documents(
                    search_index_name=search_index_name,
                    input=question,
                    embedding_function=embedding_function,
                )
                retrieved_sources = [os.path.normpath(response["source"])
                                   for response in search_response]
                retrieved_contexts = [response["chunkContext"]
                                   for response in search_response]
                retrieved_chunk_ids = [response["chunkId"]
                                   for response in search_response]
                # 2. Create prompt with the query and retrieved documents
                prompt = create_prompt(question, search_response)

                # 3. Call GPT-3 model to generate an answer
                # given the question and the retrieved documents
                response = call_llm(prompt)

                current_qa = {
                    "user_prompt": question,
                    "output_prompt": data["output_prompt"],
                    "context": data["context"],
                    "chunk_id": data["chunk_id"],
                    "source": os.path.normpath(data["source"]),
                    "root_chunk_id": data["root_chunk_id"],

                    "generated_output": response,
                    "retrieved_context": retrieved_contexts,
                    "retrieved_source": retrieved_sources,
                    "retrieved_chunk_id": retrieved_chunk_ids
                }

                generated_qa.append(current_qa)

            with open(path_to_output, "w") as f:
                json.dump(generated_qa, f)
    except Exception as e:
        print(e)

#### Generate answers using ADA + fixed size chunking - 10 min


In [77]:
%run -i ./helpers/search.ipynb

evaluation_data_path = "./output/qa/evaluation/qa_pairs_solutionops.json"

search_index_name = "fixed-size-chunks-180-30-batch-engineering-mlops-ada"
embedding_function = oai_query_embedding
path_to_output = "./output/qa/results/fixed-size-chunks-180-30-engineering-mlops-ada-2.json"

generate_answers_for_qa(evaluation_data_path, search_index_name, embedding_function, path_to_output)

KeyboardInterrupt: 

#### Generate answers using semantic chunking + open source embedding model

Note: we need to create such an index! It is not part of the previous experiments


In [13]:
%run -i ./helpers/search.ipynb

# 1. Create the new index
index_name = "semantic-chunking-eval"
embedding_path = "./output/pre-generated/embeddings/semantic-chunking-engineering-mlops-e5-small-v2.json"
vector_size = 384  # TODO: Replace with the vector size of your embedding model
create_index(index_name, vector_size)

# 2. Generate embeddings for the new chunks
# path_to_chunks_file = path_to_chunks_output
# generated_embeddings_path = f"./output/generated/{fixed_chunks_output_prefix}-embedded-{totalNumberOfDocuments}-{chunk_size}-{chunk_overlap}.json"
# generate_embeddings_for_chunks_and_save_to_file(path_to_chunks_file=path_to_chunks_output, path_to_output=generated_embeddings_path)

# 3. Upload the embeddings to the new index
upload_data(file_path=embedding_path, search_index_name=index_name)

Index: 'semantic-chunking-eval' created or updated
Uploaded 1216 documents to Index: semantic-chunking-eval


In [15]:
import json
%run -i ./helpers/search.ipynb

evaluation_data_path = "./output/qa/evaluation/qa_pairs_solutionops.json"
search_index_name = "semantic-chunking-eval"
embedding_function = intfloat_e5_small_v2_query_embedding
path_to_output = "./output/qa/results/semantic-chunking-intfloat.json"

generate_answers_for_qa(evaluation_data_path, search_index_name, embedding_function, path_to_output)


## Create a job (experiment) in Azure Machine Learning Studio


In [19]:
%%capture --no-display
subscription_id = os.environ["subscription_id"]
resource_group_name = os.environ["resource_group_name"]
workspace_name = os.environ["workspace_name"]

# experiment_name = "semantic-chunking-intfloat-e5-small-v2"
# mlflow.create_experiment(experiment_name)

'195504501528839704'

In [20]:
%%capture --no-display
# !az login
ws = Workspace.get(name=workspace_name,
                   subscription_id=subscription_id,
                   resource_group=resource_group_name)

mlflow_tracking_uri = ws.get_mlflow_tracking_uri()

## Human-Centric Metrics


To generate these metrics, we will take the [LLM as Judge approach](https://arxiv.org/pdf/2311.09476.pdf) and will use GPT-4 as the judge.


### [LLM as Judge](https://mlflow.org/docs/latest/llms/llm-evaluate/index.html#metrics-with-llm-as-the-judge)
Set the LLM model that you would like to use as judge.
Note: This steps assumes that you have previously deployed this model in your Azure OpenAI resource group.

In [ ]:
judge = "openai:/gpt-4"

### Faithfulness metric

This metric evaluates how faithful the model's generated output is to the context provided. High scores mean that the outputs contain information that is in line with the context, while low scores mean that outputs may disagree with the context (input is ignored). It uses [Likert scale](https://en.wikipedia.org/wiki/Likert_scale).

This metric is similar to [Groundedness](https://learn.microsoft.com/en-us/azure/machine-learning/prompt-flow/concept-model-monitoring-generative-ai-evaluation-metrics?view=azureml-api-2#groundedness) from Azure Machine Learning or Prompt Flow.


In [21]:
from mlflow.metrics.genai import faithfulness, EvaluationExample


# Create a good and bad example for faithfulness in the context of this problem
faithfulness_examples = [
    EvaluationExample(
        input="How do I disable MLflow autologging?",
        output="mlflow.autolog(disable=True) will disable autologging for all functions. In Databricks, autologging is enabled by default. ",
        score=2,
        justification="The output provides a working solution, using the mlflow.autolog() function that is provided in the context.",
        grading_context={
            "context": "mlflow.autolog(log_input_examples: bool = False, log_model_signatures: bool = True, log_models: bool = True, log_datasets: bool = True, disable: bool = False, exclusive: bool = False, disable_for_unsupported_versions: bool = False, silent: bool = False, extra_tags: Optional[Dict[str, str]] = None) → None[source] Enables (or disables) and configures autologging for all supported integrations. The parameters are passed to any autologging integrations that support them. See the tracking docs for a list of supported autologging integrations. Note that framework-specific configurations set at any point will take precedence over any configurations set by this function."
        },
    ),
    EvaluationExample(
        input="How do I disable MLflow autologging?",
        output="mlflow.autolog(disable=True) will disable autologging for all functions.",
        score=5,
        justification="The output provides a solution that is using the mlflow.autolog() function that is provided in the context.",
        grading_context={
            "context": "mlflow.autolog(log_input_examples: bool = False, log_model_signatures: bool = True, log_models: bool = True, log_datasets: bool = True, disable: bool = False, exclusive: bool = False, disable_for_unsupported_versions: bool = False, silent: bool = False, extra_tags: Optional[Dict[str, str]] = None) → None[source] Enables (or disables) and configures autologging for all supported integrations. The parameters are passed to any autologging integrations that support them. See the tracking docs for a list of supported autologging integrations. Note that framework-specific configurations set at any point will take precedence over any configurations set by this function."
        },
    ),
]

faithfulness_metric = faithfulness(
    model="openai:/gpt-4", examples=faithfulness_examples)

# print(faithfulness_metric)

### Relevance metric

This metric evaluates how relevant the model's generated output is with respect to both the input and the provided context. High scores mean that the model has understood the context and correct extracted relevant information from the context, while low score mean that output has completely ignored the question and the context and could be hallucinating.


In [22]:
from mlflow.metrics.genai import relevance, EvaluationExample


relevance_metric = relevance(model="openai:/gpt-35-turbo")
# print(relevance_metric)

### Similarity metric

This metric evaluates how similar the model's generated output is compared to the information in the ground_truth. High scores mean that your model outputs contain similar information as the ground_truth, while low scores mean that outputs may disagree with the ground_truth.


In [23]:
from mlflow.metrics.genai import answer_similarity
similarity_metric = answer_similarity(model="openai:/gpt-35-turbo")
# print(similarity_metric)

### Fluency metric


There is no pre-defined fluency metric in MLflow. However, we can create [custom LLM metrics](https://mlflow.org/docs/latest/llms/llm-evaluate/index.html#creating-custom-llm-evaluation-metrics).

We will define fluency in the exact same way as it is defined in [Azure SDK for Python](https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/ai/azure-ai-generative/azure/ai/generative/evaluate/pf_templates/built_in_metrics/qa/gpt_fluency_prompt.jinja2).

Therefore, fluency will measure the quality of individual sentences in the answer, and whether they are well-written and grammatically correct.


In [24]:
fluency_example_1 = mlflow.metrics.genai.EvaluationExample(
    input="What did you have for breakfast today?",
    output="Breakfast today, me eating cereal and orange juice very good.",
    justification="The answer completely lacks fluency",
    score=1,
)
fluency_example_2 = mlflow.metrics.genai.EvaluationExample(
    input="How do you feel when you travel alone?",
    output="Alone travel, nervous, but excited also. I feel adventure and like its time.",
    justification="The answer mostly lacks fluency",
    score=2,
)
fluency_example_3 = mlflow.metrics.genai.EvaluationExample(
    input="When was the last time you went on a family vacation?",
    output="Alone travel, nervous, but excited also. I feel adventure and like its time.",
    justification="The answer is partially fluent",
    score=3,
)
fluency_example_4 = mlflow.metrics.genai.EvaluationExample(
    input="What is your favorite thing about your job?",
    output="My favorite aspect of my job is the chance to interact with diverse people. I am constantly learning from their experiences and stories.",
    justification="The answer is mostly fluent",
    score=4,
)
fluency_example_5 = mlflow.metrics.genai.EvaluationExample(
    input="Can you describe your morning routine?",
    output=(
        "Every morning, I wake up at 6 am, drink a glass of water, and do some light stretching."
        "After that, I take a shower and get dressed for work. Then, I have a healthy breakfast, "
        "usually consisting of oatmeal and fruits, before leaving the house around 7:30 am.",
    ),
    justification="The answer is completely fluent",
    score=5,
)

In [49]:
fluency = mlflow.metrics.genai.make_genai_metric(
    name="fluency",
    definition=(
        "Fluency measures the quality of individual sentences in the answer, and whether they are well-written and grammatically correct. "
        "Consider the quality of individual sentences when evaluating fluency. "
        "Given the question and answer, score the fluency of the answer between one to five stars using the following rating scale:"
    ),
    grading_prompt=(
        "One star: the answer completely lacks fluency"
        "Two stars: the answer mostly lacks fluency"
        "Three stars: the answer is partially fluent"
        "Four stars: the answer is mostly fluent"
        "Five stars: the answer has perfect fluency"
    ),
    examples=[fluency_example_1, fluency_example_2,
              fluency_example_3, fluency_example_4, fluency_example_5],
    model="openai:/gpt-35-turbo",
    # parameters={"temperature": 0.0},
    # aggregations=["mean", "variance"],
    # greater_is_better=True,
)

### Coherence

There is no pre-defined coherence metric in MLflow. However, we can create [custom LLM metrics](https://mlflow.org/docs/latest/llms/llm-evaluate/index.html#creating-custom-llm-evaluation-metrics).

We will define fluency in the exact same way as it is defined in [Azure SDK for Python](https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/ai/azure-ai-generative/azure/ai/generative/evaluate/pf_templates/built_in_metrics/qa/gpt_coherence_prompt.jinja2).

Therefore, coherence of an answer will be measured by how well all the sentences fit together and sound naturally as a whole.


In [26]:
coherence_example_1 = mlflow.metrics.genai.EvaluationExample(
    input="What is your favorite indoor activity and why do you enjoy it?",
    output="I like pizza. The sun is shining.",
    justification="The answer completely lacks fluency",
    score=1,
)
coherence_example_2 = mlflow.metrics.genai.EvaluationExample(
    input="Can you describe your favorite movie without giving away any spoilers?",
    output="It is a science fiction movie. There are dinosaurs. The actors eat cake. People must stop the villain.",
    justification="The answer mostly lacks fluency",
    score=2,
)
coherence_example_3 = mlflow.metrics.genai.EvaluationExample(
    input="What are some benefits of regular exercise?",
    output="Regular exercise improves your mood. A good workout also helps you sleep better. Trees are green.",
    justification="The answer is partially fluent",
    score=3,
)
coherence_example_4 = mlflow.metrics.genai.EvaluationExample(
    input="How do you cope with stress in your daily life?",
    output="I usually go for a walk to clear my head. Listening to music helps me relax as well. Stress is a part of life, but we can manage it through some activities.",
    justification="The answer is mostly fluent",
    score=4,
)
coherence_example_5 = mlflow.metrics.genai.EvaluationExample(
    input="What can you tell me about climate change and its effects on the environment?",
    output=(
        "Climate change has far-reaching effects on the environment. "
        "Rising temperatures result in the melting of polar ice caps, contributing to sea-level rise. "
        "Additionally, more frequent and severe weather events, such as hurricanes and heatwaves, can cause disruption to ecosystems and human societies alike."
    ),
    justification="The answer is completely fluent",
    score=5,
)

In [50]:
coherence = mlflow.metrics.genai.make_genai_metric(
    name="coherence",
    definition=(
        "Coherence of an answer is measured by how well all the sentences fit together and sound naturally as a whole. "
        "Consider the overall quality of the answer when evaluating coherence. "
        "Given the question and answer, score the coherence of answer between one to five stars using the following rating scale:"
    ),
    grading_prompt=(
        "One star: the answer completely lacks coherence"
        "Two stars: the answer mostly lacks coherence"
        "Three stars: the answer is partially coherent"
        "Four stars: the answer is mostly coherent"
        "Five stars: the answer has perfect coherency"
    ),
    examples=[coherence_example_1, coherence_example_2,
              coherence_example_3, coherence_example_4, coherence_example_5],
    model="openai:/gpt-35-turbo",
    # parameters={"temperature": 0.0},
    # aggregations=["mean", "variance"],
    # greater_is_better=True,
)

In [54]:
%%capture --no-display

import json
import pandas as pd

# experiment_name = "test-experiment"
# mlflow.create_experiment(experiment_name, artifact_location="s3://your-bucket")
# TODO: "ground_truth" https://mlflow.org/docs/latest/python_api/mlflow.metrics.html#mlflow.metrics.genai.answer_similarity


def evaluate(input_path, experiment_name, run_name):
    with open(input_path, "r", encoding="utf-8") as file:
        qa_evluation_data = json.load(file)
        df = pd.DataFrame.from_records(qa_evluation_data)
        df["user_prompt"] = df["user_prompt"].astype(str)
        df["output_prompt"] = df["output_prompt"].astype(str)
        df["retrieved_context"] = df["retrieved_context"].astype(str)
        df["generated_output"] = df["generated_output"].astype(str)

        mlflow.set_tracking_uri(mlflow_tracking_uri)
        mlflow.set_experiment(experiment_name)
        with mlflow.start_run(run_name=run_name) as run:
            results = mlflow.evaluate(data=df,
                                      # predictions="output_prompt",
                                      predictions="generated_output",
                                      model_type="question-answering",
                                      extra_metrics=[
                                          faithfulness_metric,
                                          relevance_metric,
                                          coherence,
                                          fluency,
                                          #   similarity_metric,
                                        #   mlflow.metrics.latency()
                                          ],
                                      evaluator_config={
                                          "col_mapping": {
                                              "inputs": "user_prompt",  # Define the column name for the input
                                              # "context": "context",
                                              "context": "retrieved_context",
                                              #   "targets": "output_prompt"
                                          }
                                      })

            # mlflow.log_metric('toxicity', results.metrics['toxicity/v1/p90'])
            mlflow.log_metric('faithfulness_mean',
                              results.metrics['faithfulness/v1/mean'])

        return results

Evaluate the fixed-size chunking strategy with ada - took > 12 mins

In [69]:
%%capture --no-display

experiment_name = "semantic-chunking-intfloat-e5-small-v2"
mlflow.create_experiment(experiment_name)

input_path = "./output/qa/results/semantic-chunking-intfloat.json"
run_name = "4metrics"
results_semantic_chunking = evaluate(input_path, experiment_name, run_name)

print(results_semantic_chunking.metrics)

In [63]:
results_semantic_chunking.metrics

{'latency/mean': 0.0,
 'latency/variance': 0.0,
 'latency/p90': 0.0,
 'toxicity/v1/mean': 0.0005107220515492372,
 'toxicity/v1/variance': 2.3427437314922485e-06,
 'toxicity/v1/p90': 0.0006927762471605093,
 'toxicity/v1/ratio': 0.0,
 'faithfulness/v1/mean': 3.8060200668896322,
 'faithfulness/v1/variance': 1.4406326551157145,
 'faithfulness/v1/p90': 5.0,
 'relevance/v1/mean': 3.47,
 'relevance/v1/variance': 1.3824333333333334,
 'relevance/v1/p90': 5.0,
 'coherence/v1/mean': 3.530201342281879,
 'coherence/v1/variance': 0.7323093554344399,
 'coherence/v1/p90': 4.0,
 'fluency/v1/mean': 3.508417508417508,
 'fluency/v1/variance': 0.6876395832624789,
 'fluency/v1/p90': 4.0}

In [70]:
pre_generated_results = {'latency/mean': 0.0,
                         'latency/variance': 0.0,
                         'latency/p90': 0.0,
                         'toxicity/v1/mean': 0.0005107220515492372,
                         'toxicity/v1/variance': 2.3427437314922485e-06,
                         'toxicity/v1/p90': 0.0006927762471605093,
                         'toxicity/v1/ratio': 0.0,
                         'faithfulness/v1/mean': 3.8060200668896322,
                         'faithfulness/v1/variance': 1.4406326551157145,
                         'faithfulness/v1/p90': 5.0,
                         'relevance/v1/mean': 3.47,
                         'relevance/v1/variance': 1.3824333333333334,
                         'relevance/v1/p90': 5.0,
                         'coherence/v1/mean': 3.530201342281879,
                         'coherence/v1/variance': 0.7323093554344399,
                         'coherence/v1/p90': 4.0,
                         'fluency/v1/mean': 3.508417508417508,
                         'fluency/v1/variance': 0.6876395832624789,
                         'fluency/v1/p90': 4.0}

Evaluate the semantic chunking strategy with eval embedding

In [64]:
%%capture --no-display

experiment_name = "fixed-size-180-30-ada"
mlflow.create_experiment(experiment_name)

input_path = "./output/qa/results/fixed-size-chunks-180-30-engineering-mlops-ada.json"
run_name = "4metrics"
results_fixed_size_ada = evaluate(input_path, experiment_name, run_name)

In [60]:
results_fixed_size_ada.metrics

{'latency/mean': 0.0,
 'latency/variance': 0.0,
 'latency/p90': 0.0,
 'toxicity/v1/mean': 0.0002763730055691364,
 'toxicity/v1/variance': 1.1839384498113026e-07,
 'toxicity/v1/p90': 0.0005062455020379276,
 'toxicity/v1/ratio': 0.0,
 'faithfulness/v1/mean': 2.6755852842809364,
 'faithfulness/v1/variance': 2.132213286204852,
 'faithfulness/v1/p90': 5.0,
 'relevance/v1/mean': 2.276666666666667,
 'relevance/v1/variance': 1.7534555555555555,
 'relevance/v1/p90': 4.0,
 'coherence/v1/mean': 2.59,
 'coherence/v1/variance': 1.0485666666666669,
 'coherence/v1/p90': 4.0,
 'fluency/v1/mean': 2.622895622895623,
 'fluency/v1/variance': 0.9487013796778109,
 'fluency/v1/p90': 4.0}

In [71]:
pre_generated_results = {'latency/mean': 0.0,
                         'latency/variance': 0.0,
                         'latency/p90': 0.0,
                         'toxicity/v1/mean': 0.0002763730055691364,
                         'toxicity/v1/variance': 1.1839384498113026e-07,
                         'toxicity/v1/p90': 0.0005062455020379276,
                         'toxicity/v1/ratio': 0.0,
                         'faithfulness/v1/mean': 2.6755852842809364,
                         'faithfulness/v1/variance': 2.132213286204852,
                         'faithfulness/v1/p90': 5.0,
                         'relevance/v1/mean': 2.276666666666667,
                         'relevance/v1/variance': 1.7534555555555555,
                         'relevance/v1/p90': 4.0,
                         'coherence/v1/mean': 2.59,
                         'coherence/v1/variance': 1.0485666666666669,
                         'coherence/v1/p90': 4.0,
                         'fluency/v1/mean': 2.622895622895623,
                         'fluency/v1/variance': 0.9487013796778109,
                         'fluency/v1/p90': 4.0}

In [67]:
import pandas as pd
keys = ['relevance/v1/mean', 'faithfulness/v1/mean',
        'coherence/v1/mean', 'fluency/v1/mean']
semantic_chunking_eval_values = [
    results_semantic_chunking.metrics[key] for key in keys]
fixed_size_chunking_ada_values = [
    results_fixed_size_ada.metrics[key] for key in keys]
table = [semantic_chunking_eval_values, fixed_size_chunking_ada_values]
df = pd.DataFrame(table, columns=['Relevance', 'Faithfulness', 'Coherence', 'Fluency'], index=[
                  'semantic_chunking_eval', 'fixed_size_chunking_ada'])
print(df)

                         Relevance  Faithfulness  Coherence   Fluency
semantic_chunking_eval    3.470000      3.806020   3.530201  3.508418
fixed_size_chunking_ada   2.276667      2.675585   2.590000  2.622896
